# 토큰화

In [1]:
!pip install gluonnlp pandas tqdm   

!pip install mxnet

!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2

#PyTorch불러오는거
!pip install torch
!pip install --upgrade torch torchvision

# KoBERT 토크나이저& 본체
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for gluonnlp
Failed to build gluonnlp
  Running setup.py install for gluonnlp: started
  Running setup.py install for gluonnlp: finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [124 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-39
      creating build\lib.win-amd64-cpython-39\gluonnlp
      copying src\gluonnlp\base.py -> build\lib.win-amd64-cpython-39\gluonnlp
      copying src\gluonnlp\_constants.py -> build\lib.win-amd64-cpython-39\gluonnlp
      copying src\gluonnlp\__init__.py -> build\lib.win-amd64-cpython-39\gluonnlp
      creating build\lib.win-amd64-cpython-39\gluonnlp\calibration
      copying src\gluonnlp\calibration\collector.py -> build\lib.win-amd64-cpython-39\gluonnlp\calibration
      copying src\gluonnlp\calibration\__init__.py -> build\lib.win-amd64-cpython-39\gluonnlp\calibration
      creating build\lib.win-amd64-cpython-39\gluonnlp\data
      copying src\gluonnlp\data\baidu_ernie_data.py -> build\lib.win-amd64-cpytho

In [1]:
import pandas as pd

In [ ]:
!pip install plotly

     ---------------------------------------- 15.2/15.2 MB 7.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\YuEuDdem\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

from tqdm import tqdm, tqdm_notebook

from kobert import get_pytorch_kobert_model
from kobert import get_tokenizer

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel


device = torch.device("cuda:0")
model, vocab = get_pytorch_kobert_model()

ModuleNotFoundError: No module named 'kobert'

In [1]:
import torch
x = torch.rand(5, 3)
# from kobert import get_tokenizer


ModuleNotFoundError: No module named 'torch'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data=pd.read_excel('/content/drive/MyDrive/hackerthon/웰니스 대화 스크립트 데이터셋/02)웰니스_대화_스크립트_데이터셋.xlsx')

In [4]:
ffill_columns=['핵심증상','intent','keyword(임상키워드)','임상질문그룹(연세의료원제공)']
for column_name in ffill_columns:
  data[column_name].fillna(method="ffill",inplace=True)


# class_data=data.loc[data['intent']==data['intent'].unique()[0]]
# print(type(class_data['utterance'].unique()))
# print(class_data['utterance(2차) '].unique())
labeled_data=[]
for idx,intent_item in enumerate(data['intent'].unique()):
  class_data=data.loc[data['intent']==intent_item]
  for j in class_data['utterance'].unique():
    if(not pd.isnull(j)):
      labeled_data.append([j,idx])
  for j in class_data['utterance(2차) '].unique():
    if(not pd.isnull(j)):
      labeled_data.append([j,idx])

print(len(labeled_data))

23994


In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        
        temp_list=[]
        for i in dataset:
          temp_list.append(transform([i[sent_idx]]))
        self.sentences = temp_list
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [6]:
#KoBERT 다중분류기 클래스
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=19,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [7]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [8]:
#문장이 주어지면 예측하는 함수 정의
def predict(predict_sentence,original_data):
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            intents=original_data['intent'].unique()
            test_eval.append(intents[~pd.isnull(intents)][np.argmax(logits)])
        print(test_eval[0])
        print(">> 현재 화자의 심경은 " + test_eval[0] + " 입니다.")
        return test_eval[0]

In [9]:
# Setting parameters
max_len = 64 #  허용가능한 최대 문장길이 (토큰화 이후)
batch_size = 32 # 배치 크기 - 50개 데이터에서 배치가 10이면 5번의 반복으로 모든 데이터를 볼수있음
warmup_ratio = 0.1 # 전체학습중 warmup step의 비율 - warmup step은 수치적 안정성을 위해 학습초반 조금더 작은 learning_rate를 적용하는 것을의미함
num_epochs = 20 # 전체데이터 학습시키는 횟수
num_workers = 105 # 데이터를 로딩하는데 사용할 멀티프로세스 수
max_grad_norm = 1 
log_interval = 200
learning_rate =  5e-5

In [10]:
#train set, test set 나누기기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(labeled_data, test_size=0.1, random_state=0)

In [11]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [12]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [13]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=num_workers)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=num_workers)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 105 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:
#BERT 모델 불러오기
model = BERTClassifier(model,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다항항분류를 위한 대표적인 loss func
#loss_fn = nn.BCELoss() # 이항분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [15]:
# 학습시키기

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    #학습모드
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        #print(label.shape,out.shape)
        #m=nn.Softmax(dim=1)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    #평가모드
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

    
    torch.save(model,f'/content/drive/MyDrive/hackerthon/감정분류모델-민영/model{e}.pt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/675 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.9419219493865967 train acc 0.0
epoch 1 batch id 201 loss 2.7479329109191895 train acc 0.08970771144278607
epoch 1 batch id 401 loss 2.08209228515625 train acc 0.16038029925187033
epoch 1 batch id 601 loss 1.5176312923431396 train acc 0.27745424292845255
epoch 1 train acc 0.315772792022792


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/75 [00:00<?, ?it/s]

...

epoch 19 test acc 0.87


  0%|          | 0/675 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.0002865628630388528 train acc 1.0
epoch 20 batch id 201 loss 0.0003098843153566122 train acc 0.9995335820895522
epoch 20 batch id 401 loss 0.0002781819202937186 train acc 0.9995324189526185
epoch 20 batch id 601 loss 0.0003748632443603128 train acc 0.9993760399334443
epoch 20 train acc 0.9993981481481482


  0%|          | 0/75 [00:00<?, ?it/s]

epoch 20 test acc 0.8695833333333334


In [18]:
import plotly.express as px
import plotly.graph_objects as go
 
x=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
fig = go.Figure() # Figure 추가
fig.add_traces(go.Scatter(x=x, y=train_history, name='Train Accuracy', line = dict(color='royalblue', width=4))) # 선 그래프 추가.
fig.add_traces(go.Scatter(x=x, y=test_history, name='Test Accuracy', line = dict(color='firebrick', width=4))) # 선 그래프 추가.
fig.show() # 그래프 출력 이 부분은 생략해도 그래프는 출력됨.

In [25]:
while True :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    model=torch.load(f'/content/drive/MyDrive/hackerthon/감정분류모델-민영/model18.pt',map_location=device)
    predict(sentence,data)
    print("\n")

하고싶은 말을 입력해주세요 : 난 원래 사계절이 어울리지 않아 차가운 아이잖아
정신증상/자존감저하
>> 현재 화자의 심경은 정신증상/자존감저하 입니다.


하고싶은 말을 입력해주세요 : 모진 성격은 살을 에는 겨울 시린 바람 알만큼 알잖아
정신증상/분노
>> 현재 화자의 심경은 정신증상/분노 입니다.


하고싶은 말을 입력해주세요 : 내겐 냉정이 다 인걸 겨울바다 같은 심장인걸
정신증상/자존감저하
>> 현재 화자의 심경은 정신증상/자존감저하 입니다.


하고싶은 말을 입력해주세요 : 배를 띄워 다가오면 알겠지 내가 섬이 아닌 빙산인걸
정신증상/자존감저하
>> 현재 화자의 심경은 정신증상/자존감저하 입니다.


하고싶은 말을 입력해주세요 : 난 초점을 잃었지
정신증상/집중력저하
>> 현재 화자의 심경은 정신증상/집중력저하 입니다.


하고싶은 말을 입력해주세요 : 서리 낀 눈 시린 기억이 밟혀서 목젖 없이 비명만 질렀지
정신증상/분노
>> 현재 화자의 심경은 정신증상/분노 입니다.


하고싶은 말을 입력해주세요 : 아픈 상처만 남은 빙판이었지
정신증상/무기력
>> 현재 화자의 심경은 정신증상/무기력 입니다.


하고싶은 말을 입력해주세요 : 눈사태 난 듯 무너진 맘
정신증상/슬픔
>> 현재 화자의 심경은 정신증상/슬픔 입니다.


하고싶은 말을 입력해주세요 : 추스려 보니 다시 불 꺼진 밤
정신증상/슬픔
>> 현재 화자의 심경은 정신증상/슬픔 입니다.


하고싶은 말을 입력해주세요 : 너란 해가 떠도 눈 덮인 산은 녹지 않아
정신증상/외로움
>> 현재 화자의 심경은 정신증상/외로움 입니다.


하고싶은 말을 입력해주세요 : 0
